Please note: this script uses sclite to analyze the hypothesis transcripts
You can find installation instructions here https://github.com/usnistgov/SCTK

In [ ]:
import os
import subprocess
from pathlib import Path

In [ ]:
hyp_path = Path("path/to/your/data/analysis/hyp") ###
ref_path = Path("path/to/your/data/analysis/ref") ###
out_path = Path("path/to/your/data/analysis/results") ###

In [ ]:
def sort_ctm_file(path):
    with open(path, 'r', encoding='utf-8') as f:
        lines = [line.strip() for line in f if line.strip()]

    sorted_lines = sorted(lines, key=lambda x: (
        x.split()[0],            #filename
        x.split()[1],            #channel
        float(x.split()[2])      #start time
    ))

    with open(path, 'w', encoding='utf-8', newline='\n') as f:
        for line in sorted_lines:
            f.write(line + '\n')

In [39]:
def analyze_ctm_dtl(hyp_file, ref_file, out_path):
    os.makedirs(out_path, exist_ok=True)

    command = [
        "sclite",
        "-r", str(ref_file).replace("\\", "/"), "stm",
        "-h", str(hyp_file).replace("\\", "/"), "ctm",
        "-i", "rm",
        "-o", "dtl",
        "-O", str(out_path).replace("\\", "/")
    ]

    subprocess.run(command)

def analyze_ctm_sum(hyp_file, ref_file, out_path):
    os.makedirs(out_path, exist_ok=True)

    command = [
        "sclite",
        "-r", str(ref_file).replace("\\", "/"), "stm",
        "-h", str(hyp_file).replace("\\", "/"), "ctm",
        "-i", "rm",
        "-o", "sum",
        "-O", str(out_path).replace("\\", "/")
    ]

    subprocess.run(command)

In [ ]:
stm_files = ref_path.glob("*.stm")
model_names = [f.name for f in hyp_path.iterdir() if f.is_dir()] 

for model in model_names:
    model_path = hyp_path / model
    ctm_path = model_path / "ctm"

    for hyp_file in list(ctm_path.glob("*.ctm")):
        print(f"Processing file: {hyp_file.name}")
        subset = hyp_file.stem.split("_")[1]
        ref_file = ref_path / f"{subset}_reference.stm"

        sort_ctm_file(hyp_file)
        analyze_ctm_dtl(hyp_file, ref_file, out_path / model / f"{model}_{subset}")
        analyze_ctm_sum(hyp_file, ref_file, out_path / model / f"{model}_{subset}")